<a href="https://colab.research.google.com/github/john-caballero/Perfilamiento-basico/blob/main/sesion1/Laboratorio_1_%E2%80%94_Segmentaci%C3%B3n_por_Reglas%2C_GE%E2%80%93McKinsey_(9_box)_y_RFM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Laboratorio 1 — Segmentación por Reglas, GE–McKinsey (9-box) y RFM**

---

**Objetivos**

- Construir 3–6 segmentos con reglas simples.

- Priorizar segmentos con GE–McKinsey (Atractividad × Fuerza).

- Etiquetar clientes con RFM (quintiles) y proponer 2–3 acciones.

**Índice**

- Cargar datos

- Chequeos rápidos

- Segmentación por reglas

- 9-box GE–McKinsey

- RFM (quintiles + etiquetas)

- Salidas y decisiones

## 1) Cargar datos

In [ ]:
import pandas as pd, numpy as np

# Reemplaza por tu ruta:
# df = pd.read_csv("tu_archivo.csv")  # o read_excel(...)
df = df.copy()  # si ya lo cargaste antes


## 2) Chequeos rápidos

In [ ]:
num_cols = ["MONTO","PLAZO","DEF12","PD_FINAL2","MTO_DEURCCVIG_AVGU12",
            "R_MTODEURCCVIG_AVGU06_AVGU12","FRQ_ABO_GT100U12","MTO_ABO_AVGU12",
            "R_ABO_AVGU06_AVGU12","MTO_TRX_AVGU12","MTO_TRXNET_AVGU12",
            "FRQ_PASPM_GT1000U12","MTO_PAS_AVGU12","R_PASPM_U06U12"]

for c in num_cols:
    if c in df: df[c] = pd.to_numeric(df[c], errors="coerce")

print("NAs (%) top 10:\n", df.isna().mean().sort_values(ascending=False).head(10))
df[num_cols].describe(percentiles=[.33,.66,.9]).T


## 3) Segmentación por reglas (simple)

In [ ]:
# Umbrales básicos (si no hay PD_FINAL2, salta a DEF12 por grupo si aplica)

p33 = df["PD_FINAL2"].quantile(.33) if "PD_FINAL2" in df else np.nan
p66 = df["PD_FINAL2"].quantile(.66) if "PD_FINAL2" in df else np.nan

frq66 = df["FRQ_ABO_GT100U12"].quantile(.66) if "FRQ_ABO_GT100U12" in df else np.nan

abo66 = df["MTO_ABO_AVGU12"].quantile(.66) if "MTO_ABO_AVGU12" in df else np.nan

monto66 = df["MONTO"].quantile(.66) if "MONTO" in df else np.nan


In [ ]:
cond1 = ("PD_FINAL2" in df) & ("FRQ_ABO_GT100U12" in df) & ("MTO_ABO_AVGU12" in df) \
        and (df["PD_FINAL2"] <= p33) & (df["FRQ_ABO_GT100U12"] >= frq66) & (df["MTO_ABO_AVGU12"] >= abo66)

cond2 = ("PD_FINAL2" in df) & ("MONTO" in df) \
        and (df["PD_FINAL2"].between(p33, p66, inclusive="right")) & (df["MONTO"] >= monto66)

cond3 = ("PD_FINAL2" in df) & ("FRQ_ABO_GT100U12" in df) \
        and (df["PD_FINAL2"] > p66) & (df["FRQ_ABO_GT100U12"] < df["FRQ_ABO_GT100U12"].quantile(.33))

cond4 = ("FLG_REENGANCHE_FINAL" in df) and (df["FLG_REENGANCHE_FINAL"] == 1)



In [ ]:
df["SEG_REGLAS"] = np.select(
    [cond1, cond2, cond3, cond4],
    ["S1_Prime_Engaged_TicketMedioAlto", "S2_Medio_TicketAlto", "S3_AltoRiesgo_BajaFrecuencia", "S4_Reenganche"],
    default="S5_Resto"
)


In [ ]:
seg_view = df.groupby("SEG_REGLAS").agg(
    n=("SEG_REGLAS","size"),
    pct=("SEG_REGLAS", lambda s: 100*len(s)/len(df)),
    def12=("DEF12","mean"),
    pdm=("PD_FINAL2","mean"),
    ticket_med=("MONTO","median"),
    abo_med=("MTO_ABO_AVGU12","median"),
    frq_abos=("FRQ_ABO_GT100U12","median")
).reset_index().sort_values("def12")
seg_view


## 4) 9-box GE–McKinsey

In [ ]:
# A = promedio min–max de [pct, ticket_med, abo_med]
# F = promedio min–max de [(1-def12), frq_abos, (1-reeng)]

reeng = df.groupby("SEG_REGLAS")["FLG_REENGANCHE_FINAL"].mean() if "FLG_REENGANCHE_FINAL" in df else None
ge = seg_view.merge(reeng.rename("reeng"), left_on="SEG_REGLAS", right_index=True, how="left") if reeng is not None else seg_view.assign(reeng=np.nan)

def mm(x):
    x = x.astype(float)
    return (x - x.min())/(x.max()-x.min()) if x.max()!=x.min() else x*0

A = pd.concat([mm(ge["pct"]), mm(ge["ticket_med"]), mm(ge["abo_med"])], axis=1).mean(axis=1)
F = pd.concat([mm(1-ge["def12"]), mm(ge["frq_abos"]), mm(1-ge["reeng"].fillna(0))], axis=1).mean(axis=1)

ge["Atractividad"], ge["Fuerza"] = A, F

Qa, Qf = ge["Atractividad"].quantile([1/3,2/3]), ge["Fuerza"].quantile([1/3,2/3])
def tercil(x,q): return 1 if x<q.iloc[0] else (3 if x>q.iloc[1] else 2)
ge["A_3"] = ge["Atractividad"].apply(lambda x: tercil(x,Qa))
ge["F_3"] = ge["Fuerza"].apply(lambda x: tercil(x,Qf))
ge["Estrategia_GE"] = np.where((ge["A_3"]==3)&(ge["F_3"]==3),"Grow (Invertir)",
                        np.where((ge["A_3"]==3)|(ge["F_3"]==3),"Selectivo","Harvest / Mantener"))

ge[["SEG_REGLAS","Atractividad","Fuerza","A_3","F_3","Estrategia_GE","def12","ticket_med","abo_med","frq_abos","pct"]]


## 5) RFM (quintiles + etiquetas)

In [ ]:
def q5(s):
    try:
        return pd.qcut(s.rank(method="first"), 5, labels=[1,2,3,4,5]).astype(float)
    except Exception:
        return pd.Series(np.nan, index=s.index)



In [ ]:
df["R_q"] = q5(df["R_ABO_AVGU06_AVGU12"]) if "R_ABO_AVGU06_AVGU12" in df else np.nan
df["F_q"] = q5(df["FRQ_ABO_GT100U12"]) if "FRQ_ABO_GT100U12" in df else np.nan
df["M_q"] = q5(df["MTO_ABO_AVGU12"]) if "MTO_ABO_AVGU12" in df else np.nan
df["RFM_score"] = df[["R_q","F_q","M_q"]].sum(axis=1)



In [ ]:
def tag(r,f,m):
    if r>=4 and f>=4 and m>=4: return "RFM_PremiumActivo"
    if r<=2 and f<=2 and m<=2: return "RFM_BajoValor_Latente"
    if f>=4 and m>=4:          return "RFM_AltoValor_Retener"
    if r<=2 and f>=4:          return "RFM_Antiguo_Frecuente_Reactivar"
    return "RFM_Medio"



In [ ]:
df["RFM_cluster"] = [tag(r,f,m) for r,f,m in df[["R_q","F_q","M_q"]].to_numpy()]



In [ ]:
rfm_view = df.groupby("RFM_cluster").agg(
    n=("RFM_cluster","size"),
    pct=("RFM_cluster", lambda s: 100*len(s)/len(df)),
    def12=("DEF12","mean"),
    R=("R_q","median"), F=("F_q","median"), M=("M_q","median"),
    abo_med=("MTO_ABO_AVGU12","median")
).reset_index().sort_values("def12")
rfm_view


## 6) Salidas y decisiones

- seg_view: lectura por segmento (tamaño, DEF12, ticket, abonos, frecuencia).

- ge: ubicación 9-box + Estrategia_GE.

- rfm_view: clusters RFM con DEF12.

- Tarea en clase: elegir 2–3 decisiones (precio, verificación, límite, campaña).

---

# Gracias por completar este laboratorio!

---
